In [1]:
'''
Section [1]
'''

import numpy
numpy.set_printoptions(linewidth=180)
pp = lambda x: print(numpy.array(x))

SBOX = (9, 10, 11, 1, 2, 0, 20, 18, 19, 3, 4, 5, 22, 23, 21, 14, 12, 26, 24, 25, 13, 16, 17, 15, 8, 6, 7)

F.<a> = GF(3^3, modulus=x^3 + x^2 + 2)

tri = [F.fetch_int(i) for i in range(27)]
lng = {x: x.integer_representation() for x in tri}

def dif(s):
    s2 = {tri[i]:tri[s[i]] for i in s}
    out = []
    for i in tri:
        layer = [0 for _ in range(27)]
        for p in tri:
            layer[lng[s2[p + i] - s2[p]]] += 1
        out.append(layer)
    return out

ddt = dif(SBOX)
pp(ddt)
chance = [max(i) for i in ddt]
print(chance)



[[27  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 23  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  2  0  0]
 [ 0  0 23  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  2  0]
 [ 0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  2  0  0  0 23  0  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0 23  0  0  0  0  0  0]
 [ 0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  2 23  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  2  0  0  0 23  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  2 23  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0]
 [ 0  0  0  0  0  0  2  0  0  0 23  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0]
 [ 0  0  0  0  0  0  0  2  0  0  0  2  0  0  0  0  0  0  0  0  0 23  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  2  2  0  0  0  0  0  0  0  0  0  0  0  0 23  0  0  0  0]
 [ 0  0  0  0  0  0  2  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0 23  0

In [2]:
'''
Section [2]
'''
ROUNDS = 6
def mald(tup, a, b):
    out = 0
    for i, j in enumerate(tup):
        out2 = 0
        for k, l in enumerate(j):
            out2 += l*a^k
        out += out2*b^i
    return out

def unmald(pol):
    return tuple(tuple(i.polynomial().list()) for i in pol.list())

G.<b> = PolynomialRing(F)
mm = ((2, 0, 1), (1, 2, 0), (0, 2, 1), (2, 0, 1))
MM = mald(mm, a, b)
H.<c> = G.quotient(MM) # Setting up for mix columns

cc = ((1, 2, 0), (2, 0, 1), (1, 1, 1))
CC = mald(cc, a, c)
DD = CC.inverse_mod(MM)
assert CC*DD == 1

def add(a, b):
    return [i + j for i, j in zip(a, b)]

def sub(a):
    b = []
    c = []
    for i in a:
        inp = lng[i]
        row = ddt[inp]
        col = row.index(chance[inp])
        out = tri[col] 
        b.append(out)
        c.append(chance[inp]/27)
    return b, c

def mix(a):
    b = [0 for _ in range(9)]

    for i in range(3):
        we = a[i::3]
        we = we[0] + we[1]*c + we[2]*c^2
        we *= CC

        b[i::3] = we.list()
    return b

def row(a):
    return [a[i] for i in (0, 1, 2, 4, 5, 3, 8, 6, 7)]

lst = {i:([0, 0, 0] + i.polynomial().list()) for i in F}

def calc(x): # skips add()
    hist = []
    for _ in range(ROUNDS):
        x, y = sub(x)
        hist += y
        x = row(x)
        x = mix(x)
    x, y = sub(x)
    hist += y
    x = row(x)
    return x, prod(hist).numerical_approx(), sum(i < 1 for i in hist), hist


Waa

In [3]:
import os
print(os.listdir('.'))

['.solution.ipynb.sage-jupyter2', '.smc', '.2021-08-03-003824.ipynb.sage-jupyter2', 'lol.txt', 'dump.txt', '.snapshots', '.bash_history', 'output.txt', '.jupyter-blobs-v0.db', '2021-08-03-003824.ipynb', '.ccache', '.2021-08-03-003824.ipynb-0.term', '.sage', 'solution.ipynb', '.cache', '.bash_profile', '.gitexcludes', '.bashrc', '.ssh', '.gitconfig']


In [4]:
with open("./output.txt", "r") as f:
    chal = eval(f.read()[217:])

In [5]:
print(chal[0])

b'\x05\xfeG[\xb5)'


In [6]:
int_to_byt = lambda x: x.to_bytes((x.bit_length() + 7) // 8, "big")
byt_to_int = lambda x: int.from_bytes(x, byteorder="big")

def byt_to_tri(x):
    x = [int(i) for i in Integer(byt_to_int(x)).str(3)]
    x.reverse()
    return [F(x[i:i + 3]) for i in range(0, 27, 3)]

In [7]:
chal = [byt_to_tri(x) for x in chal]

In [8]:
print(chal[0])

[a^2 + a + 1, 2*a, a, 2*a^2 + 2*a, 2*a^2 + 2*a, 2, 2*a^2 + 2*a + 2, 2*a + 2, 2*a^2 + a + 2]


In [9]:
zadd = lambda x, y: x + y
zsub = lambda x, y: x - y
def zlip(a, b, k = zadd):
    return [k(i, j) for i, j in zip(a, b)]

def lint(a):
    out = 0
    for i in a[::-1]:
        out *= 27
        out += lng[i]
    return out

def ltri(a):
    x = [int(i) for i in Integer(a).str(3)]
    x.reverse()
    return [F(x[i:i + 3]) for i in range(0, 27, 3)]

print(ltri(lint([2, 1, 0, 0, 0, 0, 0, 0, 0])))

[2, 1, 0, 0, 0, 0, 0, 0, 0]


In [10]:
print(byt_to_tri(b"3"))
print(ord("5"))

[2*a^2 + 2*a, 1, 0, 0, 0, 0, 0, 0, 0]
53


In [11]:
print(eval("a**2 + 1"))
from sage.misc.persist import dumps, loads

a^2 + 1


In [12]:
nom = [(1, 2, a*a), (1, 0, a*a), (1, 2, a*a), (1, 2, a*a)]
print(list(set(nom)))
print((*nom, 1, 2))

[(1, 2, a^2), (1, 0, a^2)]
((1, 2, a^2), (1, 0, a^2), (1, 2, a^2), (1, 2, a^2), 1, 2)


In [13]:
from itertools import product, islice
safe = []
undup = lambda x: list(set(x))
try:
    with open("dumps.txt", "rb") as f:
        safe = loads(f.read())
    nom = safe.pop()
    nom2 = lng[nom[0]] * 27^2 + lng[nom[1]] * 27 + lng[nom[2]]
    print(f"Starting at {nom2} => {nom}...")
    if safe:
        print(f"> {safe[-1]}")
    print()
    kek = islice(product(tri, tri, tri), nom2, None)
except Exception as e:
    print(e)
    print("Something went wrong?????")
    print("^^^")
    kek = product(tri, tri, tri)
    next(kek)

try:
    for i in kek:
        safe2 = []
        guess = (*i, 0, 0, 0, 0, 0, 0)
        out = calc(guess)
        for j, k in enumerate(chal):
            nj = lint(zlip(guess, ltri(j)))
            nk = chal[nj]
            #assert zlip(ltri(nj), ltri(j), zsub) == guess
            if out[0] == zlip(nk, k, zsub):
                safe2.append((j, nj, guess, tuple(out[0])))
        if safe2:
            safe += safe2
except Exception as e:
    print(f"Error: {e}")
    choice = input("You wish to save?? [y/n]")
    if choice == "n":
        raise SystemExit
except KeyboardInterrupt:
    print("Escaping loop...")


print(f"Stopped at {i}")
safe = undup(safe)
safety = dumps(safe + [i])
with open("dumps.txt", "wb+") as f:
    f.write(safety)

print("Done!?!?!")

[Errno 2] No such file or directory: 'dumps.txt'
Something went wrong?????
^^^


Escaping loop...
Stopped at (0, a, 2*a^2 + 1)
Done!?!?!


In [14]:
print(safe[0])
print(safe[-1])
print(len(safe))

(6765, 18429, (0, 0, a^2 + 2*a + 1, 0, 0, 0, 0, 0, 0), (a^2 + 2*a + 1, 2, 2*a + 1, 2*a^2 + 2*a + 1, 2*a^2 + a, 2*a + 2, a^2 + 2, 2*a^2 + 2*a + 2, a + 1))
(11009, 10955, (0, 1, 0, 0, 0, 0, 0, 0, 0), (2*a^2 + 2*a, a^2 + a + 2, a^2 + 2*a + 2, 2*a + 1, 2*a^2 + 2*a + 2, a^2 + 2, 2*a^2 + a + 2, a^2 + a + 1, 2*a^2))
687


In [15]:
test = [a, 2*a, 0, a^2, 2, a^2 + 2, 1, 2*a^2, a]

def usub(a):
    b = []
    c = []
    for i in a:
        out = lng[i]
        col = [_[out] for _ in ddt]
        prob = max(col)
        row = col.index(prob)
        inp = tri[row]
        b.append(inp)
        c.append(prob/27)
    return b, c

assert usub(sub(test)[0])[0] == test

def umix(a):
    b = [0 for _ in range(9)]

    for i in range(3):
        we = a[i::3]
        we = we[0] + we[1]*c + we[2]*c^2
        we *= DD

        b[i::3] = we.list()
    return b

assert umix(mix(test)) == test

def urow(a):
    return [a[i] for i in (0, 1, 2, 5, 3, 4, 7, 8, 6)]

assert urow(row(test)) == test

ussub = {tri[SBOX[lng[i]]]:i for i in tri}
ssub = {i:tri[SBOX[lng[i]]] for i in tri}

tsub = lambda x: [ssub[i] for i in x]
utsub = lambda x: [ussub[i] for i in x]

In [16]:
def vroom():
    scores = [[0] * 27 for _ in range(9)]
    for i in safe:
        i1, i2, din, dout = i
        o1, o2 = chal[i1], chal[i2]
        dmid, o1, o2 = usub(urow(dout))[0], urow(o1), urow(o2)
        for pos, kpos in enumerate((0, 1, 2, 5, 3, 4, 7, 8, 6)):
            expect = dmid[pos]
            for key in tri:
                k1, k2 = ussub[o1[pos] - key], ussub[o2[pos] - key]
                if k2 - k1 == expect:
                    scores[kpos][lng[key]] += 1
    final = []
    for score in scores:
        k = tri[score.index(max(score))]
        print(f"{str(k):>16} {score}")
        final.append(k)
    return final

In [17]:
print(f"-> {str(vroom())}")

     a^2 + a + 2 [566, 631, 594, 552, 613, 582, 596, 581, 578, 595, 560, 609, 583, 567, 687, 581, 546, 601, 609, 581, 567, 581, 593, 581, 631, 573, 582]
       2*a^2 + a [600, 563, 582, 560, 589, 596, 583, 621, 607, 573, 578, 621, 596, 572, 563, 587, 602, 566, 580, 606, 568, 687, 605, 580, 557, 572, 564]
   2*a^2 + a + 2 [593, 573, 584, 580, 580, 582, 627, 581, 588, 568, 627, 579, 590, 580, 592, 579, 582, 560, 564, 561, 589, 591, 577, 687, 565, 609, 588]
               a [586, 595, 555, 687, 606, 590, 592, 595, 575, 581, 579, 563, 551, 578, 568, 612, 643, 584, 601, 591, 643, 583, 567, 582, 556, 575, 587]
   a^2 + 2*a + 1 [594, 561, 586, 627, 537, 586, 553, 567, 625, 610, 549, 571, 599, 588, 575, 585, 687, 551, 569, 598, 627, 583, 614, 575, 573, 611, 605]
             a^2 [579, 577, 596, 576, 575, 589, 575, 559, 634, 687, 597, 570, 599, 577, 614, 583, 571, 560, 559, 576, 591, 584, 634, 630, 566, 573, 569]
           a + 1 [599, 588, 591, 584, 687, 583, 562, 576, 608, 591, 589, 572, 566,

In [18]:
def subtracc(a, b):
    return [x - y for x, y in zip(a, b)]

In [19]:
print(safe[0])
print(safe[-1])

(6765, 18429, (0, 0, a^2 + 2*a + 1, 0, 0, 0, 0, 0, 0), (a^2 + 2*a + 1, 2, 2*a + 1, 2*a^2 + 2*a + 1, 2*a^2 + a, 2*a + 2, a^2 + 2, 2*a^2 + 2*a + 2, a + 1))
(11009, 10955, (0, 1, 0, 0, 0, 0, 0, 0, 0), (2*a^2 + 2*a, a^2 + a + 2, a^2 + 2*a + 2, 2*a + 1, 2*a^2 + 2*a + 2, a^2 + 2, 2*a^2 + a + 2, a^2 + a + 1, 2*a^2))


In [20]:
def mega_usbox(spec):
    keys = [[0] * 27 for _ in range(9)]
    spec = [(i, j, usub(k)[0]) for i, j, k in spec]
    for pos in range(9):
        for key in tri:
            cnt = sum([ussub[j[pos] - key] - ussub[i[pos] - key] == k[pos] for i, j, k in spec])
            keys[pos][lng[key]] = cnt
    key = [tri[i.index(max(i))] for i in keys]
    for i in keys:
        if len(set(i)) == 1:
            print(f"Data set too small (Difference of 0 found) {keys}.")
    return spec, key

def mega_vroom():
    spec = [(chal[i], chal[j], list(k)) for i, j, _, k in safe]
    final = []

    spec = [(urow(i), urow(j), urow(k)) for i, j, k in spec]
    spec, key = mega_usbox(spec)
    spec = [(utsub(subtracc(i, key)), utsub(subtracc(j, key)), k) for i, j, k in spec]
    final.append(row(key))

    for _ in range(ROUNDS):
        print(_)
        spec = [(urow(umix(i)), urow(umix(j)), urow(umix(k))) for i, j, k in spec]
        spec, key = mega_usbox(spec)
        spec = [(utsub(subtracc(i, key)), utsub(subtracc(j, key)), k) for i, j, k in spec]
        final.append(mix(row(key)))

    last = [(subtracc(j[0], chal[safe[i][0]]), subtracc(j[1], chal[safe[i][0]])) for i, j in enumerate(spec)]
    print(f"Sanity check... {last[0][0] == last[0][1] == last[-1][1]}")
    final.append(last[0][0])
    return final[::-1]

In [21]:
kek = mega_vroom()

0


1


2


3


4


Data set too small (Difference of 0 found) [[687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687], [640, 581, 654, 650, 644, 643, 593, 655, 653, 658, 591, 663, 656, 687, 623, 591, 650, 618, 615, 635, 687, 631, 624, 635, 583, 685, 640], [607, 595, 601, 618, 593, 559, 606, 551, 651, 651, 609, 557, 547, 591, 541, 568, 645, 573, 564, 568, 578, 577, 687, 537, 631, 627, 595], [687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687], [633, 569, 662, 625, 614, 616, 663, 640, 643, 655, 687, 635, 642, 629, 636, 619, 658, 614, 687, 622, 654, 588, 611, 663, 616, 667, 637], [554, 573, 586, 614, 566, 527, 651, 631, 618, 530, 609, 631, 593, 588, 560, 557, 687, 585, 553, 548, 614, 687, 595, 645, 565, 576, 573], [687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687

Data set too small (Difference of 0 found) [[687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687], [595, 635, 585, 665, 687, 653, 631, 619, 656, 656, 613, 596, 621, 664, 655, 654, 658, 687, 687, 620, 640, 616, 687, 607, 603, 577, 618], [632, 587, 687, 643, 557, 645, 520, 565, 566, 599, 535, 579, 687, 572, 587, 557, 609, 554, 573, 595, 569, 599, 584, 536, 568, 687, 627], [687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687], [687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687], [687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687], [687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687, 687

In [22]:
for i in kek:
    print(i)

[a^2 + a + 2, 2*a^2, a^2 + 2*a + 2, 2*a^2, 2*a^2 + 2*a + 1, a^2 + a + 1, 2*a^2 + a + 2, a^2 + 2*a + 1, 2*a^2 + 2]
[0, 2*a^2 + 1, a + 2, 0, 2*a, 2*a^2 + 1, 0, a^2 + 2*a + 2, 2*a^2 + 2*a + 2]
[2*a^2 + a, 2*a^2 + 2*a, a + 1, 2*a^2 + a + 2, a^2 + 1, 2, a^2 + 1, 1, 2*a^2 + a + 1]
[a^2 + a, 2*a^2 + 2*a + 2, 2*a + 1, 0, a^2, a^2 + 2*a, a, a^2 + 2*a, 1]
[a, a^2 + 2*a + 1, a + 2, a^2, a, 0, 2*a^2 + a, 2*a^2 + 2, 2*a + 1]
[2*a^2 + a + 1, 0, 2*a^2 + a + 1, 2*a^2 + 2*a + 1, a^2 + 2*a, 2*a^2 + a + 2, 2*a^2 + 1, 2*a^2 + a + 1, 2*a^2 + 2*a + 1]
[1, 2*a^2 + 2*a + 1, 2*a^2 + 2*a + 1, 2*a^2 + a + 1, a^2 + 2*a, 2*a^2 + a + 2, 2*a^2 + 2*a + 1, a^2 + 2*a, 0]
[a^2 + a + 2, 2*a^2 + a, 2*a^2 + a + 2, a, a^2 + 2*a + 1, a^2, a + 1, a + 2, 2*a + 1]


In [23]:
print(safe[-1])
five = kek[-5:] # change this depending on when error starts
print(five)

(11009, 10955, (0, 1, 0, 0, 0, 0, 0, 0, 0), (2*a^2 + 2*a, a^2 + a + 2, a^2 + 2*a + 2, 2*a + 1, 2*a^2 + 2*a + 2, a^2 + 2, 2*a^2 + a + 2, a^2 + a + 1, 2*a^2))
[[a^2 + a, 2*a^2 + 2*a + 2, 2*a + 1, 0, a^2, a^2 + 2*a, a, a^2 + 2*a, 1], [a, a^2 + 2*a + 1, a + 2, a^2, a, 0, 2*a^2 + a, 2*a^2 + 2, 2*a + 1], [2*a^2 + a + 1, 0, 2*a^2 + a + 1, 2*a^2 + 2*a + 1, a^2 + 2*a, 2*a^2 + a + 2, 2*a^2 + 1, 2*a^2 + a + 1, 2*a^2 + 2*a + 1], [1, 2*a^2 + 2*a + 1, 2*a^2 + 2*a + 1, 2*a^2 + a + 1, a^2 + 2*a, 2*a^2 + a + 2, 2*a^2 + 2*a + 1, a^2 + 2*a, 0], [a^2 + a + 2, 2*a^2 + a, 2*a^2 + a + 2, a, a^2 + 2*a + 1, a^2, a + 1, a + 2, 2*a + 1]]


In [24]:
def slic(x):
    y = sum(x, [])
    z = [y[i:i+3] for i in range(0, len(y), 3)]
    return z
fiv = slic(five)
print(fiv)

size = ROUNDS + 2 # 8
ksize = ROUNDS - 1
print(size)

def expp(five):
    key = [[0]*3 for _ in range(size*3)] # 24
    key[-len(five):] = five
    print(key[9])
    for i in range(size*3-len(five), -1, -1): #24
        if i % ksize == 0: # 5
            print("Wtf")
            t = key[i + ksize]
            t[0] -= a^(i // ksize)
            l = tsub(key[i + ksize - 1])
            l = l[1:] + l[:1]
            print(l)
            key[i] = subtracc(t, l)
        else:
            key[i] = subtracc(key[i + ksize], key[i + ksize - 1])
        print(i, key[i])
    return key

yay = expp(fiv)
for i in yay:
    print(i)

[[a^2 + a, 2*a^2 + 2*a + 2, 2*a + 1], [0, a^2, a^2 + 2*a], [a, a^2 + 2*a, 1], [a, a^2 + 2*a + 1, a + 2], [a^2, a, 0], [2*a^2 + a, 2*a^2 + 2, 2*a + 1], [2*a^2 + a + 1, 0, 2*a^2 + a + 1], [2*a^2 + 2*a + 1, a^2 + 2*a, 2*a^2 + a + 2], [2*a^2 + 1, 2*a^2 + a + 1, 2*a^2 + 2*a + 1], [1, 2*a^2 + 2*a + 1, 2*a^2 + 2*a + 1], [2*a^2 + a + 1, a^2 + 2*a, 2*a^2 + a + 2], [2*a^2 + 2*a + 1, a^2 + 2*a, 0], [a^2 + a + 2, 2*a^2 + a, 2*a^2 + a + 2], [a, a^2 + 2*a + 1, a^2], [a + 1, a + 2, 2*a + 1]]
8
[a^2 + a, 2*a^2 + 2*a + 2, 2*a + 1]
9 [a^2 + a, 2*a^2 + 2*a + 2, 2*a + 1]
8 [a^2 + 2*a, 2*a^2 + 2*a + 2, 2*a + 1]
7 [0, 1, a + 1]
6 [a, 2*a, 2*a^2 + a + 1]
Wtf
[2*a + 1, 2*a^2, 2*a^2 + a + 1]
5 [2, 2*a^2, 2*a^2 + a + 2]
4 [2*a, 0, 0]
3 [a^2 + 2*a, 2*a^2 + 2*a + 1, a]
2 [2*a, a + 1, a^2]
1 [a + 1, a^2 + 2*a, 2]
Wtf
[a^2, a^2, 2*a^2 + 2]
0 [2*a^2 + 1, a^2, a]
[2*a^2 + 1, a^2, a]
[a + 1, a^2 + 2*a, 2]
[2*a, a + 1, a^2]
[a^2 + 2*a, 2*a^2 + 2*a + 1, a]
[2*a, 0, 0]
[1, 2*a^2, 2*a^2 + a + 2]
[a, 2*a, 2*a^2 + a + 1]
[0

In [25]:
wee = sum(yay[:5], [])
print(wee)
weed= 0
for i in wee[::-1]:
    print(i)
    weed *= 27
    weed += i.integer_representation()
key = int_to_byt(int(weed))
print(key)

from hashlib import sha512

hsh = sha512(key).digest()
print(hsh)

def byte_xor(a, b):
    return bytes([i ^^ j for i, j in zip(a, b)])

flag = byte_xor(b"v\x17\xed'\xad:\xa2t\xaf\x8c\x8bT7\x9af\xc0\xf4\x92w\x82WR\xb4md\xafh_\xcc\xe9\xa6d:7f;.\xd0h\xc1\x12DU\x02&\x91Z\xbf\x9a\x05\x08A\xc4\xb9\xde\xed\x90\xda/f\xfd\x07\xf6I", hsh)
print(flag)

[2*a^2 + 1, a^2, a, a + 1, a^2 + 2*a, 2, 2*a, a + 1, a^2, a^2 + 2*a, 2*a^2 + 2*a + 1, a, 2*a, 0, 0]
0
0
2*a
a
2*a^2 + 2*a + 1
a^2 + 2*a
a^2
a + 1
2*a
2
a^2 + 2*a
a + 1
a
a^2
2*a^2 + 1
b'\x0c\xcdiD\x8cc\x18\xf2'
b'\x04v\x9fD\xd9\\\xd9A\xdb\xbc\xfb\x0bg\xf5\x15\xf7\xc5\xfcN\xa3\x08f\xd6]\x11\xdb7k\xa1\xd9\xe8#ebS\x1a\x0f\x8fY\xf657\nl\x16\xe5\x05\x88\xefKFv\x9b\x8a\xbc\xd4\xf3\xb9\x17\x03\xcc5\xc24'
b"rarctf{5t0p_Pos71n9!_4b0ut_4m0NG_U5!!_17's_n0t_7uNN7_3b9cc8e124}"
